<a href="https://colab.research.google.com/github/rainermesi/wiki_Parse/blob/master/wiki_word_gen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Prep

Import modules

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [51]:
import numpy as np
import pandas as pd
from pandas.core.common import flatten
import random
import re
from collections import defaultdict
from collections import Counter
import unicodedata
! pip install BeautifulSoup4
from bs4 import BeautifulSoup

Read in wikipedia corpus

In [111]:
wiki_full_corpus = open(r'/content/drive/My Drive/DATA/Wikipedia/etwiki_latest/wiki_et.txt','r',encoding='utf-8').read()
wiki_word_list = wiki_full_corpus.lower().split()

Read in Tammsaare corpus

In [3]:
! pip install ebooklib
import ebooklib
from ebooklib import epub

     |████████████████████████████████| 112kB 2.9MB/s 
  Created wheel for ebooklib: filename=EbookLib-0.17.1-cp36-none-any.whl size=38163 sha256=fc9c570c4df76b3a4586c9b9028d069409b7b5877ba063c8e71d5d647f92dbcf
  Stored in directory: /root/.cache/pip/wheels/84/11/01/951369cbbf8f96878786a1f4da68bd7ac19a5d945b38e03d54
Successfully built ebooklib


In [4]:
book = epub.read_epub(r'/content/drive/My Drive/DATA/Wikipedia/etwiki_latest/Anton_Hansen_Tammsaare_Tode_ja_oigus_I.epub')

In [5]:
book_corpus = []

for i in book.get_items_of_type(ebooklib.ITEM_DOCUMENT):
  book_corpus.append(i.get_content())

In [145]:
def epub_to_text(corpus):
  blacklist = ['[document]','noscript','header','html','meta','head','input','script']
  output_str = ''
  output_list = []
  for i in corpus:
    soup = BeautifulSoup(i,'html.parser')
    text = soup.find_all(text=True)
    for t in text: 
      if t.parent.name not in blacklist:
        output_list.append(unicodedata.normalize("NFKD",t).strip())
  output_list = [i.split() for i in output_list]
  output_list = list(flatten(output_list))
  output_list_copy = [i for i in output_list if len(i) > 2]
  output_list = [i.replace(',','') for i in output_list_copy]
  output_list = [i.replace("'",'') for i in output_list]
  output_list = [i.replace("«",'') for i in output_list]
  output_list = [i.replace("»",'') for i in output_list]
  output_list = [i.lower() for i in output_list]
  return output_list

In [146]:
book_word_list = epub_to_text(book_corpus)

In [147]:
book_word_list

['see',
 'e-raamat',
 'skaneeritud',
 'koostatud',
 'tartu',
 'linnaraamatukogus',
 'tartu',
 '2011',
 'see',
 'oli',
 'läinud',
 'aastasaja',
 'kolmanda',
 'veerandi',
 'lõpul.',
 'päike',
 'lähenes',
 'silmapiirile',
 'seistes',
 'sedavõrd',
 'madalas',
 'enam',
 'ulatunud',
 'valgustama',
 'mäkke',
 'ronivat',
 'hobust',
 'kes',
 'puutelgedega',
 'vankrit',
 'vedas',
 'vankril',
 'istuvat',
 'noort',
 'naist',
 'ega',
 'ligi',
 'kolmekümnelist',
 'meest',
 'kes',
 'kõndis',
 'vankri',
 'kõrval.',
 'varsti',
 'jõudsid',
 'teelised',
 'mäerinnakul',
 'nii',
 'kõrgele',
 'päikeses',
 'helendama',
 'lõid',
 'mehe',
 'nägu',
 'laiavõitu',
 'tugevate',
 'lõuapäradega',
 'terassilmadega',
 'lühikese',
 'kuid',
 'tiheda',
 'musta',
 'habemega',
 'naise',
 'nukrad',
 'silmad',
 'look',
 'hobuse',
 'kikkis',
 'kõrvadega',
 'pea.',
 'seal',
 'ongi',
 'see',
 'vargamäe',
 'lausus',
 'mees',
 'näitas',
 'käega',
 'üle',
 'soo',
 'järgmise',
 'väljamäe',
 'poole',
 'kus

## Analysis

What are most popular words in corpus?

In [148]:
pd_word_series = pd.Series(book_word_list)

In [134]:
pd_word_series.value_counts().head(50)

KUI          2490
AGA          2262
OLI          2028
SIIS         1627
SEE          1314
NAGU         1285
TEMA         1211
OMA          1151
MIS          1112
SEST         1017
ANDRES        940
SEDA          849
OLEKS         816
NII           807
VÕI          803
NÕNDA        735
ÜTLES        668
NÜÜD        639
PEARU         635
MARI          592
EGA           579
AINULT        569
KES           533
VEEL          528
NAD           510
IKKA          474
JUBA          469
KÕIK         464
KAS           452
VASTAS        434
KÜLL         416
ISE           414
VARGAMÄE     403
NING          394
OLID          385
SELLE         375
MINA          367
POLNUD        362
MIDAGI        354
PÄRAST       333
PEALE         331
TEDA          330
KÜSIS        326
TAL           319
OMETI         314
PIDI          311
MITTE         307
ÄRA          301
EHK           299
MUIDU         293
dtype: int64

How long are words in corpus?

In [102]:
def word_len_df_gen(in_list):
    count_list = [len(item) for item in in_list]
    count_df = pd.DataFrame.from_dict(Counter(count_list).items())
    count_df.sort_values(by=1,ascending=False)
    return count_df

In [170]:
word_len_df = word_len_df_gen(pd_word_series)
word_len_df.head(50)

,0,1
0,3,18914
1,8,10382
2,11,2579
3,9,7513
4,5,25647
5,17,90
6,4,27458
7,7,16849
8,6,25971
9,12,1602


Create a dictionay for the graph and parse the wiki corpus

In [136]:
def create_graph_dict(corpus):
  graphdict = defaultdict(lambda:defaultdict(int))
  for word in corpus:
    prev_letter = word[0]
    for letter in word[1:]:
      graphdict[prev_letter][letter] += 1
      prev_letter = letter
  return graphdict

In [150]:
graph_dict = create_graph_dict(pd_word_series)

Clean up the graph dictionary

In [165]:
def graph_cleanup(graph):
  abc = ['A', 'a', 'B', 'b', 'D', 'd', 'E', 'e', 'F', 'f', 'G', 'g', 'H', 'h', 'I', 'i', 'J', 'j', 'K', 'k', 'L', 'l', 'M', 'm', 'N', 'n', 'O', 'o', 'P', 'p', 'R', 'r', 'S', 's', 'Š', 'š', 'Z', 'z', 'Ž', 'ž', 'T', 't', 'U', 'u', 'V', 'v', 'Õ', 'õ', 'Ä', 'ä', 'Ö', 'ö', 'Ü', 'ü']
  abc = list(dict.fromkeys(i.lower() for i in abc))
  # clean primary keys
  tempgraph = dict((k, graph[k]) for k in abc if k in graph) 
  # clean nested key value pairs
  for letter in abc:
    try:
      for item in tempgraph[letter].copy():
        if item not in abc:
          del tempgraph[letter][item]
    except:
        print(letter,'not in corpus as key, skipping letter')
  return tempgraph

In [166]:
graph_dict = graph_cleanup(graph_dict)

f  not in corpus as key
š  not in corpus as key
z  not in corpus as key
ž  not in corpus as key
õ  not in corpus as key
ä  not in corpus as key
ö  not in corpus as key
ü  not in corpus as key


Traverse the graph_dict and create new words

In [138]:
def traverse_graph(graph, word_len=3, start_node=None):
  """Returns a list of words from a randomly weighted walk."""
  if word_len <= 0:
    return []
  
  # If not given, pick a start node at random.
  if not start_node:
    start_node = random.choice(list(graph.keys()))
  
  
  weights = np.array(
      list(graph[start_node].values()),
      dtype=np.float64)
  # Normalize letter counts to sum to 1. Create % weights for each letter.
  weights /= weights.sum()

  # Pick next letter using weighted distribution.
  choices = list(graph[start_node].keys())
  chosen_letter = np.random.choice(choices, None, p=weights)
  
  # recursively build a word until word_len = 0
  return [chosen_letter] + traverse_graph(
      graph, word_len=word_len-1,
      start_node=chosen_letter)

In [171]:
for i in range(10): 
  print(''.join(traverse_graph(graph_dict,word_len=random.choice(word_len_df[0]))))

ugakuisuudauistuhtta
aava
amama
telearit
sehimitesessetamimohela
amakehatsisistugajui
udalkubravagiloo
sstaragidindale
umelmidrulesaredmeev
orukiha
